In [0]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
import sys

import warnings
warnings.filterwarnings("ignore")

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

import numpy as np
import pandas as pd
import time

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D

%matplotlib inline
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
import io
import zipfile
from PIL import Image

Using TensorFlow backend.


In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

### **Loading Train Data**

In [5]:
file_list = drive.ListFile({'q': "'1IsZnTUo2pSXDH00Xq42Ye8JJe768XUtB' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: selfie_test_data.zip, id: 1Kx5Xgz1eN69c0OvgLPvWniXFdBmZ9gRi
title: adience_split.zip, id: 11GsvwXPQ4UGSSU2bILA_QeXXThHkFpy4
title: selfie_test_data, id: 1YcwE0YD_fFJxGvxPoQK7STEH58eGQXeZ
title: adience_split, id: 13HFhSYmArdZXIcf9jN78crzxFHl8Rxdt


In [0]:
data_downloaded = drive.CreateFile({'id': '11GsvwXPQ4UGSSU2bILA_QeXXThHkFpy4'})
data_downloaded.GetContentFile('adience_split.zip')

In [0]:
data = zipfile.ZipFile('adience_split.zip', 'r')
dir_list = data.namelist()

In [0]:
data.extractall('adience_split/')

In [9]:
!ls

adience_split  adience_split.zip  datalab


### Model

In [10]:
! pip install keras_vggface

  Running setup.py bdist_wheel for keras-vggface ... - done
  Stored in directory: /content/.cache/pip/wheels/72/aa/01/eb7baeb2f6e2d2f0d2aabddb5f01d57fa22fbd019ee2799bf5
Successfully built keras-vggface


In [13]:
# model from scratch (mfs)
# using the caffe model params as a guide
from keras.engine import  Model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.optimizers import SGD, Adam
from keras_vggface.vggface import VGGFace

#custom parameters
nb_class = 2
hidden_dim = 512

vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
vgg_model.layers[0].trainable = False 

last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)

layer_count = 0
for layer in custom_vgg_model.layers:
	layer_count = layer_count+1
for l in range(layer_count-3):
	custom_vgg_model.layers[l].trainable=False

custom_vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

### Training on Entire Dataset

In [14]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
source_path = 'adience_split/adience_split/'

train_generator = datagen.flow_from_directory(
        source_path+'train',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

val_generator = datagen.flow_from_directory(
        source_path+'val',
        target_size=(224, 224),
        batch_size=32, 
        shuffle = True)
#         class_mode='binary')

test_generator = datagen.flow_from_directory(
        source_path+'test',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

Found 8950 images belonging to 2 classes.
Found 3837 images belonging to 2 classes.
Found 5481 images belonging to 2 classes.


In [15]:
history = custom_vgg_model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=50,
        validation_data=val_generator,
        validation_steps=80,
        shuffle = True)

Epoch 1/50
100/100 [==============================] - 93s 927ms/step - loss: 0.3239 - acc: 0.8525 - val_loss: 0.2366 - val_acc: 0.9020
Epoch 2/50
100/100 [==============================] - 84s 845ms/step - loss: 0.2183 - acc: 0.9125 - val_loss: 0.1842 - val_acc: 0.9285
Epoch 3/50
100/100 [==============================] - 84s 838ms/step - loss: 0.1847 - acc: 0.9268 - val_loss: 0.1703 - val_acc: 0.9375
Epoch 4/50
 63/100 [=================>............] - ETA: 15s - loss: 0.1061 - acc: 0.9551

100/100 [==============================] - 81s 814ms/step - loss: 0.1075 - acc: 0.9574 - val_loss: 0.1690 - val_acc: 0.9344
Epoch 5/50
100/100 [==============================] - 82s 816ms/step - loss: 0.0946 - acc: 0.9684 - val_loss: 0.1866 - val_acc: 0.9426
Epoch 6/50
100/100 [==============================] - 82s 816ms/step - loss: 0.0826 - acc: 0.9712 - val_loss: 0.2215 - val_acc: 0.9391
Epoch 7/50
 81/100 [=======================>......] - ETA: 7s - loss: 0.0353 - acc: 0.9904

100/100 [==============================] - 81s 813ms/step - loss: 0.0446 - acc: 0.9856 - val_loss: 0.2145 - val_acc: 0.9305
Epoch 8/50
100/100 [==============================] - 81s 812ms/step - loss: 0.0538 - acc: 0.9797 - val_loss: 0.2067 - val_acc: 0.9426
Epoch 9/50
100/100 [==============================] - 82s 815ms/step - loss: 0.0453 - acc: 0.9869 - val_loss: 0.2032 - val_acc: 0.9367
Epoch 10/50
 84/100 [========================>.....] - ETA: 6s - loss: 0.0161 - acc: 0.9940

100/100 [==============================] - 82s 824ms/step - loss: 0.0171 - acc: 0.9934 - val_loss: 0.2490 - val_acc: 0.9406
Epoch 11/50
100/100 [==============================] - 81s 815ms/step - loss: 0.0578 - acc: 0.9781 - val_loss: 0.1973 - val_acc: 0.9418
Epoch 12/50
100/100 [==============================] - 82s 824ms/step - loss: 0.0294 - acc: 0.9897 - val_loss: 0.2177 - val_acc: 0.9410
Epoch 13/50
 83/100 [=======================>......] - ETA: 7s - loss: 0.0157 - acc: 0.9944

100/100 [==============================] - 81s 809ms/step - loss: 0.0167 - acc: 0.9941 - val_loss: 0.2759 - val_acc: 0.9395
Epoch 14/50
100/100 [==============================] - 81s 811ms/step - loss: 0.0183 - acc: 0.9950 - val_loss: 0.3036 - val_acc: 0.9434
Epoch 15/50
100/100 [==============================] - 81s 812ms/step - loss: 0.0055 - acc: 0.9991 - val_loss: 0.3050 - val_acc: 0.9453
Epoch 16/50
 83/100 [=======================>......] - ETA: 7s - loss: 0.0134 - acc: 0.9940

...

100/100 [==============================] - 81s 806ms/step - loss: 0.0039 - acc: 0.9991 - val_loss: 0.3411 - val_acc: 0.9473
Epoch 38/50
100/100 [==============================] - 81s 810ms/step - loss: 0.0075 - acc: 0.9972 - val_loss: 0.4508 - val_acc: 0.9305
Epoch 39/50
100/100 [==============================] - 82s 822ms/step - loss: 0.0103 - acc: 0.9972 - val_loss: 0.2815 - val_acc: 0.9531
Epoch 40/50
 83/100 [=======================>......] - ETA: 7s - loss: 0.0011 - acc: 1.0000

100/100 [==============================] - 82s 820ms/step - loss: 9.6136e-04 - acc: 1.0000 - val_loss: 0.3244 - val_acc: 0.9496
Epoch 41/50
100/100 [==============================] - 81s 813ms/step - loss: 0.0045 - acc: 0.9981 - val_loss: 0.3171 - val_acc: 0.9484
Epoch 42/50
100/100 [==============================] - 81s 812ms/step - loss: 3.5290e-04 - acc: 1.0000 - val_loss: 0.3384 - val_acc: 0.9496
Epoch 43/50
 79/100 [======================>.......] - ETA: 8s - loss: 2.5442e-04 - acc: 1.0000

100/100 [==============================] - 82s 816ms/step - loss: 3.0516e-04 - acc: 1.0000 - val_loss: 0.3653 - val_acc: 0.9500
Epoch 44/50
100/100 [==============================] - 81s 815ms/step - loss: 3.6360e-05 - acc: 1.0000 - val_loss: 0.3875 - val_acc: 0.9496
Epoch 45/50
100/100 [==============================] - 81s 811ms/step - loss: 2.1034e-04 - acc: 1.0000 - val_loss: 0.4491 - val_acc: 0.9488
Epoch 46/50
 76/100 [=====================>........] - ETA: 10s - loss: 1.1962e-05 - acc: 1.0000

100/100 [==============================] - 81s 810ms/step - loss: 1.1448e-05 - acc: 1.0000 - val_loss: 0.4160 - val_acc: 0.9520
Epoch 47/50
100/100 [==============================] - 81s 808ms/step - loss: 4.6535e-06 - acc: 1.0000 - val_loss: 0.4214 - val_acc: 0.9527
Epoch 48/50
100/100 [==============================] - 80s 804ms/step - loss: 0.0045 - acc: 0.9994 - val_loss: 0.3987 - val_acc: 0.9449
Epoch 49/50
 76/100 [=====================>........] - ETA: 9s - loss: 1.4737e-04 - acc: 1.0000 

100/100 [==============================] - 81s 807ms/step - loss: 1.5154e-04 - acc: 1.0000 - val_loss: 0.3588 - val_acc: 0.9488
Epoch 50/50
100/100 [==============================] - 81s 809ms/step - loss: 0.0036 - acc: 0.9997 - val_loss: 0.3909 - val_acc: 0.9434


In [0]:
custom_vgg_model.save('vggf_train_full.h5')

In [0]:
import pickle

with open('vggHistoryDict', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [0]:
cd ../../

/jet/prs/workspace


In [0]:
pwd

'/jet/prs/workspace'

In [34]:
custom_vgg_model.predict_generator(test_generator)

array([[4.5621413e-04, 9.9954373e-01],
       [1.0000000e+00, 2.2820428e-08],
       [9.9999988e-01, 7.3189206e-08],
       ...,
       [4.6054681e-23, 1.0000000e+00],
       [1.0000000e+00, 5.1509946e-13],
       [1.6056726e-12, 1.0000000e+00]], dtype=float32)

In [40]:
custom_vgg_model.evaluate_generator(train_generator, pickle_safe=True)

[0.00400182354513798, 0.9985474860335195]

In [41]:
custom_vgg_model.evaluate_generator(val_generator, pickle_safe=True)

[0.3491943102125901, 0.9478759446086938]

In [39]:
custom_vgg_model.evaluate_generator(test_generator, pickle_safe=True)

[0.4086071009749553, 0.9366903849771218]

In [22]:
ls

adience_split/  adience_split.zip  datalab/  vggf_train_full.h5  vggHistoryDict


In [0]:
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [30]:
uploaded = drive.CreateFile({'title': 'vggf_train_full.h5'})
uploaded.SetContentFile('vggf_train_full.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1ruksJ7jhTupMiUUO6MHijc7FwlNT3Dtg


In [31]:
uploaded = drive.CreateFile({'title': 'vggHistoryDict'})
uploaded.SetContentFile('vggHistoryDict')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1i5sSFNdNb35Z_F5dh4IJg2NwpPZlpye_
